In [2]:
import json
import plotly
import pandas as pd

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from flask import Flask
from flask import render_template, request, jsonify
from plotly.graph_objs import Bar
from plotly.graph_objs import Figure
from plotly.graph_objs import Pie

import joblib
from sqlalchemy import create_engine
from werkzeug.serving import run_simple

In [3]:
app = Flask(__name__)

def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [4]:
# load data
engine = create_engine('sqlite:///./data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)

# load model
model = joblib.load("./model/classifier.pkl")

In [5]:
# index webpage displays cool visuals and receives user input text for model
@app.route('/')
@app.route('/index')
def index():
    
    # extract data needed for visuals
    # TODO: Below is an example - modify to extract data for your own visuals
   
    # graph 1
    genre_counts = df.groupby('genre').count()['message']
    genre_names = list(genre_counts.index)
    
    # graph 2 & 3
    category_names = list(df.columns)[4:]
    category_counts = []
    for name in category_names:
        category_count = df[name].values.sum()  # count number of 1 (i.e. category identified for the message) per category
        category_counts.append(category_count)
    
    # create visuals
    # TODO: Below is an example - modify to create your own visuals
    graphs = [
        {
            'data': [
                Bar(
                    x=genre_names,
                    y=genre_counts
                )
            ],

            'layout': {
                'title': 'Distribution of Message Genres',
                'yaxis': {
                    'title': "Count"
                },
                'xaxis': {
                    'title': "Genre"
                }
            }
        },
        {
            'data': [
                Bar(
                    x=category_names,
                    y=category_counts
                )
            ],

            'layout': {
                'title': 'Distribution of Message Categories',
                'yaxis': {
                    'title': "Count"
                },
                'xaxis': {
                    'title': "Category"
                }
            }
        },
        {
            'data': [
                Pie(labels=category_names, values=category_counts)
            ],

            'layout': {
                'title': 'Proportion of Message Categories',
            }
        }
        
    ]
    
    # encode plotly graphs in JSON
    ids = ["graph-{}".format(i) for i, _ in enumerate(graphs)]
    graphJSON = json.dumps(graphs, cls=plotly.utils.PlotlyJSONEncoder)
    
    # render web page with plotly graphs
    return render_template('master.html', ids=ids, graphJSON=graphJSON)


# web page that handles user query and displays model results
@app.route('/go')
def go():
    # save user input in query
    query = request.args.get('query', '') 

    # use model to predict classification for query
    classification_labels = model.predict([query])[0]
    classification_results = dict(zip(df.columns[4:], classification_labels))

    # This will render the go.html Please see that file. 
    return render_template(
        'go.html',
        query=query,
        classification_result=classification_results
    )

In [7]:
run_simple('localhost', 3000, app)

 * Running on http://localhost:3000
Press CTRL+C to quit
127.0.0.1 - - [18/Dec/2022 19:21:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 19:21:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Dec/2022 19:23:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 19:23:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 19:23:59] "GET /go?query=flood+in+my+area HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 19:25:11] "GET / HTTP/1.1" 200 -


![](./img/dash.png)

![](./img/dash-response.png)